In [1]:
import pandas as pd
import numpy as np 
import string
import math
#sentences to words
import re
from nltk.tokenize import TreebankWordTokenizer
#paras to sentences
from nltk.tokenize import sent_tokenize
#stopwords
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
%%time
data_user = pd.read_csv("users.csv")

Wall time: 678 ms


In [3]:
#Checking and dropping rows having null at About me user column
data_user.isnull().sum()
data_user = data_user.dropna(subset=['_AboutMe'])
print('Total users with about me information',data_user['_AboutMe'].count())
data_user = data_user[['_Id','_AboutMe']]
data_user.head(2)

Total users with about me information 20800


_Id                                           _AboutMe
0   -1  <p>Hi, I'm not really a person.</p>\r\n\r\n<p>...
1    1  \r\n\r\n<p>Developer at Stack Overflow focusin...

In [4]:
aboutMe_dict = data_user.set_index('_Id').T.to_dict('list')
#aboutMe_dict, user Id as key and about me content as value

In [5]:
aboutMe_list= list(aboutMe_dict.values())
#aboutMe_dict

In [6]:
#small set - list of 10 dictionary value
#aboutMe_list = aboutMe_list[:50]

In [7]:
#(1) filtering hyperlinks and 
#(2) remove numbers and convert text into lowercase
#(3) remove ascii character and special character (except #)
#(4) sentence into words
#(5) filtering punctuations
#(6) stop-words for english 

In [8]:
%%time
processed_text = []
punch = string.punctuation
stop_words_e = stopwords.words('english')

for i in range(len(aboutMe_list)):    
    # (1)
    cleanr = re.compile('<.*?>')
    content = re.sub(cleanr, ' ', aboutMe_list[i][0])
    content = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '', content, flags=re.MULTILINE)
    content = content.translate(str.maketrans('', '', string.punctuation))
    
    #(2)
    content = re.sub(r'\d+', '', content)
    content = content.lower()
    
    #(3)
    #removing non-ascii characters
    content = re.sub(r'[^\x00-\x7f]',r' ',content)
    #removing special characters
    content = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", content)
   
    #(4)
    content = TreebankWordTokenizer().tokenize(content)
    content.sort()
    
    #(5)
    content = [words for words in content if words not in punch]
    
    #(6)
    content = [words for words in content if words not in stop_words_e]
    
    #if len(content) > 1:
    processed_text.append(content)
        
print(len(processed_text))

20800
Wall time: 5.11 s


In [9]:
#processed_text, list of sentence tokens

In [10]:
#TF(term) 
# of times the term appears in document / total # of terms in document
def computeRTFDict(sentence):
    TFDict = {}
    for word in sentence:
        if word in TFDict:
            TFDict[word] += 1
        else:
            TFDict[word] = 1
    #Computes tf for each word           
    for word in TFDict:
        TFDict[word] = TFDict[word] / len(sentence)
    return TFDict

In [11]:
allDocTF=[]
for i in processed_text:
    data=computeRTFDict(i)
    allDocTF.append(data)

In [12]:
#TF of all documents
allDocTF

[{'abandoned': 0.023809523809523808,
  'anonymous': 0.023809523809523808,
  'answers': 0.023809523809523808,
  'attention': 0.023809523809523808,
  'background': 0.023809523809523808,
  'clean': 0.023809523809523808,
  'community': 0.023809523809523808,
  'deleted': 0.023809523809523808,
  'downvotes': 0.023809523809523808,
  'edits': 0.023809523809523808,
  'every': 0.023809523809523808,
  'get': 0.047619047619047616,
  'gets': 0.023809523809523808,
  'helps': 0.023809523809523808,
  'hi': 0.023809523809523808,
  'hour': 0.023809523809523808,
  'im': 0.047619047619047616,
  'keep': 0.023809523809523808,
  'like': 0.023809523809523808,
  'nobody': 0.023809523809523808,
  'old': 0.023809523809523808,
  'permanently': 0.023809523809523808,
  'person': 0.023809523809523808,
  'poke': 0.023809523809523808,
  'posts': 0.023809523809523808,
  'process': 0.023809523809523808,
  'questions': 0.07142857142857142,
  'randomly': 0.023809523809523808,
  'really': 0.023809523809523808,
  'remove': 

In [13]:
#calculate total no of document and total no of document with word in it  
def computeCountDict():
    countDict = {}
    for sentences in allDocTF:
        for word in sentences:
            if word in countDict:
                countDict[word] += 1
            else:
                countDict[word] = 1
    return countDict

#Stores the user count dictionary
countDict = computeCountDict()

In [14]:
#countDict

In [15]:
#Computation of IDF
def computeIDFDict():
    idfDict = {}
    for word in countDict:
        idfDict[word] = math.log(len(processed_text) / countDict[word])
    return idfDict 
  #Stores the idf dictionary
idfDict = computeIDFDict()

In [16]:
#idfDict

In [17]:
#computation of TF-IDF
def computeTFIDFDict(userTFDict):
    userTFIDFDict = {}
    #For each word in the review, we multiply its tf and its idf.
    for word in userTFDict:
        userTFIDFDict[word] = userTFDict[word] * idfDict[word]
    return userTFIDFDict

#Stores the TF-IDF dictionaries
tfidfDict = [computeTFIDFDict(user) for user in allDocTF]

In [18]:
#constructing vector, matrix an array of vectors, where each vector represents a user about me
#The vector will be a list of frequencies for each unique word in the dataset—the TF-IDF value 
#if the word is in the about_me, or 0.0 otherwise.

In [19]:
#Create a list of unique words
wordDict = sorted(countDict.keys())
def computeTFIDFVector(sentence):
    tfidfVector = [0.0] * len(wordDict)

# For each unique word, if it is in the review, store its TF-IDF value.
    for i, word in enumerate(wordDict):
        if word in sentence:
            tfidfVector[i] = sentence[word]
    return tfidfVector

In [ ]:
%%time
tfidfVector = [computeTFIDFVector(sentence) for sentence in tfidfDict]

In [ ]:
tfidfVector

In [ ]:
  def dot_product(vector_x, vector_y):
    dot = 0.0
    for e_x, e_y in zip(vector_x, vector_y):
        dot += e_x * e_y
    return dot

In [ ]:
 def magnitude(vector):
    mag = 0.0
    for index in vector:
        mag += math.pow(index, 2)
    return math.sqrt(mag)

In [ ]:
user_similarity = dot_product(tfidfVector[0], tfidfVector[6])/ magnitude(tfidfVector[0]) * magnitude(tfidfVector[6])
user_similarity

In [ ]:
%%time
maximum = -1
maxisimilar={}
review = []
similarityDict = {} 
temp = -1
k=0
length = len(tfidfVector)
for i in tfidfVector:
    if(maximum!=-1):
        maxisimilar[str(temp)+","+str(k)] = maximum   
    maximum = 0
    temp+=1
    for j in range(temp+1,length):
        if(magnitude(i) * magnitude(tfidfVector[j])!=0):        
            data = dot_product(i, tfidfVector[j])/ magnitude(i) * magnitude(tfidfVector[j]) 
            if(maximum<data):
                maximum=data
                k=j
        else:
            data = 0
        review.append(data)
        similarityDict[str(temp)+","+str(j)] = data  

In [ ]:
#similarity of each user with its most similar fit
maxisimilar

In [ ]:
#for full data set, similarity of every pair
similarityDict

In [ ]:
#most similar pair out of whole data set
#numbers are indexes of dictionary
sorted_similarUser = sorted(similarityDict.items(), key=lambda x: x[1], reverse=True)
sorted_similarUser
sorted_similarUser[:10]

In [ ]:
#list most similar half of each us
mostsimilarUser = []
mostsimilarUser.append(maxisimilar)
mostsimilarUser

In [ ]:
#aboutMe_dict.keys()
index=[]
for i in range(0,len(sorted_similarUser[:10])):
    index.append(sorted_similarUser[i][0].split(","))
    #print(sorted_similarUser[i][0])
print(index)